In [1]:
import numpy as np
import pandas as pd
import xlrd
import os

In [2]:
df = pd.read_html(r"C:\Users\chongs\Monthly Yield calculator\V6\Z_SGS_ANALYSIS_PATTERN.html", skiprows = 2, thousands = '.', decimal = ',')[0]
col_name = pd.read_html(r"C:\Users\chongs\Monthly Yield calculator\V6\Z_SGS_ANALYSIS_PATTERN.html")[2]

col1 = col_name.iloc[0,:]
col2 = col_name.iloc[1,:]
col2.iloc[4:] = col1.shift(+2)   
df.columns = col2.values
df.iloc[:,4:] = df.iloc[:,4:].convert_objects(convert_numeric=True)

C:\Users\chongs\Anaconda3\envs\mypython\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  


In [5]:
col2.iloc[4:] = col1.shift(+2)   

In [6]:
col2

0                      Plant
1                 Workcenter
2               Tabia Family
3              Window family
4                  Raw Cons.
5                Prim. Cons.
6     R024Glass Cons. in F.P
7               In/Out Delta
8             R124Good Prod.
9                Qual. Prod.
10                    Reinj.
11                    Scraps
12               Scrap Float
13             Scraps rework
14     R212Scrap Float disc.
15          Yield1 (w/o Rwk)
16             Yield1 Budget
17              Q122Rework A
18    R004Yield2 (incl. Rwk)
19              Labour hours
20         C005Working Hours
21        C10xDowntime Hours
22                   C20xOAD
23       C311 Opt Prod T (H)
24                   C009OEE
25     C006Utilization Hours
26                 UH Budget
27           C11x% Downtimes
28           Cad1 (Cons./WH)
29           Cad2 (Cons./UH)
30                   Good/UH
31            Good/UH Budget
32                  Good/HMO
33                     WH/UH
Name: 1, dtype

### df = df.astype(str).apply(lambda x: x.str.replace(".","").str.replace(",","."))
###df = df.astype(str)
###df.iloc[:,4:]=df.iloc[:,4:].astype(float)


In [4]:
# 각 설비별 실적

In [5]:
cl = ['R024Glass Cons. in F.P','R124Good Prod.','In/Out Delta','Qual. Prod.','Q122Rework A', 'Scraps', 'Scrap Float','Scraps rework','Labour hours', 'C20xOAD','C005Working Hours', 'C006Utilization Hours','C10xDowntime Hours']
dfp = df.pivot_table(values=cl, index=['Workcenter','Window family'], aggfunc=np.sum).reindex(cl,axis = 1)

In [6]:
# Sunroof_sum

work_center = ['2BT1','2BT2','2FG8','3EP3','3EP1']

dic_new = {}

for i in work_center:
    a = 'multi-panels 1 (fron' in dfp.loc[i].index
    b = 'multi-panels 2 (back' in dfp.loc[i].index
    c = "Lamellas' sunroof 0" in dfp.loc[i].index

    if a == True and b == True and c == True:
        dic_new[i] = dfp.loc[i].loc['multi-panels 1 (fron'] + dfp.loc[i].loc['multi-panels 2 (back'] + dfp.loc[i].loc["Lamellas' sunroof 0"]

    elif a == False and b == True and c == True:
        dic_new[i] = dfp.loc[i].loc['multi-panels 2 (back'] + dfp.loc[i].loc["Lamellas' sunroof 0"]  
    elif a == True and b == False and c == True:
        dic_new[i] = dfp.loc[i].loc['multi-panels 1 (fron'] + dfp.loc[i].loc["Lamellas' sunroof 0"]
    elif a == True and b == True and c == False:
        dic_new[i] = dfp.loc[i].loc['multi-panels 1 (fron'] + dfp.loc[i].loc['multi-panels 2 (back']
    elif a == True and b == False and c == False:
        dic_new[i] = dfp.loc[i].loc['multi-panels 1 (fron']
    elif a == False and b == True and c == False:
        dic_new[i] = dfp.loc[i].loc['multi-panels 2 (back']
    elif a == False and b == False and c == True:
        dic_new[i] = dfp.loc[i].loc["Lamellas' sunroof 0"]
    else:
        dic_new[i] = None
roof_sum = pd.DataFrame(dic_new).transpose()


In [7]:
index_in = ['roof','roof','roof','roof','roof']
roof_index = list(zip(roof_sum.index,index_in))
roof_index = pd.MultiIndex.from_tuples(roof_index)
new_roof = pd.DataFrame(roof_sum.values,roof_index,roof_sum.columns)

In [8]:
#add roof index in 
dfp = dfp.append(new_roof)

In [9]:
def total_input(a, b):
    try :
        ans = (-1)*a - b
    
    except:
        
        ans = None
        print('error occured')
    return ans

   
dfp['Total_input'] = dfp.apply(lambda row: total_input(row['R024Glass Cons. in F.P'], row['Scrap Float']), axis=1)


def prod_q(a, b, c):
    try:
        ans = a-b-c
        
    except:
        ans = None
        print('error occured')
        
    return ans

dfp['Current_q'] = dfp.apply(lambda row: prod_q(row['Qual. Prod.'], row['Q122Rework A'], row['Scraps rework']), axis = 1)
dfp['Q_to_Good'] = dfp['Current_q']*75/100

def total_good(a, b, c,d):
    try:
        ans = a + b + c- d 
        
    except:
        
        ans = None
        print('error occured')
        
    return ans

dfp['Total_Good'] = dfp.apply(lambda row : total_good(row['R124Good Prod.'], row['Q122Rework A'], row['Q_to_Good'], row['In/Out Delta']), axis = 1)
def machine_yield(a, b):
    try:
        ans = a/b*100
        
    except:
        ans = None
        print('error occured')
        
    return ans

dfp['Machine_Yield'] = dfp.apply(lambda row : machine_yield(row['Total_Good'], row['Total_input']), axis = 1)

C:\Users\chongs\Anaconda3\envs\datacontrol\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: divide by zero encountered in double_scalars


In [10]:
dfp

R024Glass Cons. in F.P  R124Good Prod.  \
Workcenter Window family                                                  
-139126    -163749                               3941.0          2551.0   
2BT1       -35526                                1677.0             0.0   
           Result                              -35526.0         36485.0   
           backlite                            -27081.0         28075.0   
           multi-panels 2 (back                 -8445.0          8410.0   
2BT2       -19630                                  22.0             0.0   
           Result                              -19630.0         19646.0   
           multi-panels 1 (fron                -19630.0         19646.0   
2FG8       -25851                                 850.0             0.0   
           Lamellas' sunroof 0                  -7144.0          7085.0   
           Result                              -50474.0         50648.0   
           front door                          -10710.0         11350.0   
           rear door                           -32620.0         32213.0   
3EP1       -33156                                -320.0          2252.0   
           Lamellas' sunroof 0                      0.0           -52.0   
           Result                              -33156.0         30191.0   
           backlite                            -33156.0         30243.0   
3EP3       -24963                                 -19.0          1689.0   
           Result                              -24963.0         22497.0   
           multi-panels 1 (fron                -14558.0         13191.0   
           multi-panels 2 (back                -10405.0          9306.0   
Result     -163749                             159467.0             0.0   
2BT1       roof                                 -8445.0          8410.0   
2BT2       roof                                -19630.0         19646.0   
2FG8       roof                                 -7144.0          7085.0   
3EP1       roof                                     0.0           -52.0   
3EP3       roof                                -24963.0         22497.0   

                                 In/Out Delta  Qual. Prod.  Q122Rework A  \
Workcenter Window family                                                   
-139126    -163749                        0.0        900.0        521.14   
2BT1       -35526                     36485.0          0.0        103.78   
           Result                      1677.0          0.0          0.00   
           backlite                    1677.0          0.0          0.00   
           multi-panels 2 (back           0.0          0.0          0.00   
2BT2       -19630                     19646.0          0.0        100.08   
           Result                        22.0          0.0          0.00   
           multi-panels 1 (fron          22.0          0.0          0.00   
2FG8       -25851                     50648.0          0.0        101.41   
           Lamellas' sunroof 0           18.0          0.0          0.00   
           Result                       850.0          0.0          0.00   
           front door                   821.0          0.0          0.00   
           rear door                     11.0          0.0          0.00   
3EP1       -33156                     30191.0          0.0         94.40   
           Lamellas' sunroof 0            0.0         52.0          0.00   
           Result                      -320.0       2252.0       1109.00   
           backlite                    -320.0       2200.0       1109.00   
3EP3       -24963                     22497.0          0.0         92.18   
           Result                       -19.0       1689.0        515.00   
           multi-panels 1 (fron           0.0       1107.0        371.00   
           multi-panels 2 (back         -19.0        582.0        144.00   
Result     -163749                     3941.0       2551.0          0.00   
2BT1       roof                  

### Input() 의 결과 값은 String 이다.

In [31]:
dfp.at[('3EP1','backlite'),'Total_Good'] = dfp.at[('3EP1','backlite'),'Total_Good'] + int(input())

-360


In [11]:
dfp.at[('3EP3','roof'),'Total_Good'] = dfp.at[('3EP3','roof'),'Total_Good'] + int(input())

-100




##  Set_value 로 value 대체하는 법 
### dfp.loc['3EP1'].set_value('backlite','Total_Good', a + int(input()))

In [12]:
dfp

R024Glass Cons. in F.P  R124Good Prod.  \
Workcenter Window family                                                  
-139126    -163749                               3941.0          2551.0   
2BT1       -35526                                1677.0             0.0   
           Result                              -35526.0         36485.0   
           backlite                            -27081.0         28075.0   
           multi-panels 2 (back                 -8445.0          8410.0   
2BT2       -19630                                  22.0             0.0   
           Result                              -19630.0         19646.0   
           multi-panels 1 (fron                -19630.0         19646.0   
2FG8       -25851                                 850.0             0.0   
           Lamellas' sunroof 0                  -7144.0          7085.0   
           Result                              -50474.0         50648.0   
           front door                          -10710.0         11350.0   
           rear door                           -32620.0         32213.0   
3EP1       -33156                                -320.0          2252.0   
           Lamellas' sunroof 0                      0.0           -52.0   
           Result                              -33156.0         30191.0   
           backlite                            -33156.0         30243.0   
3EP3       -24963                                 -19.0          1689.0   
           Result                              -24963.0         22497.0   
           multi-panels 1 (fron                -14558.0         13191.0   
           multi-panels 2 (back                -10405.0          9306.0   
Result     -163749                             159467.0             0.0   
2BT1       roof                                 -8445.0          8410.0   
2BT2       roof                                -19630.0         19646.0   
2FG8       roof                                 -7144.0          7085.0   
3EP1       roof                                     0.0           -52.0   
3EP3       roof                                -24963.0         22497.0   

                                 In/Out Delta  Qual. Prod.  Q122Rework A  \
Workcenter Window family                                                   
-139126    -163749                        0.0        900.0        521.14   
2BT1       -35526                     36485.0          0.0        103.78   
           Result                      1677.0          0.0          0.00   
           backlite                    1677.0          0.0          0.00   
           multi-panels 2 (back           0.0          0.0          0.00   
2BT2       -19630                     19646.0          0.0        100.08   
           Result                        22.0          0.0          0.00   
           multi-panels 1 (fron          22.0          0.0          0.00   
2FG8       -25851                     50648.0          0.0        101.41   
           Lamellas' sunroof 0           18.0          0.0          0.00   
           Result                       850.0          0.0          0.00   
           front door                   821.0          0.0          0.00   
           rear door                     11.0          0.0          0.00   
3EP1       -33156                     30191.0          0.0         94.40   
           Lamellas' sunroof 0            0.0         52.0          0.00   
           Result                      -320.0       2252.0       1109.00   
           backlite                    -320.0       2200.0       1109.00   
3EP3       -24963                     22497.0          0.0         92.18   
           Result                       -19.0       1689.0        515.00   
           multi-panels 1 (fron           0.0       1107.0        371.00   
           multi-panels 2 (back         -19.0        582.0        144.00   
Result     -163749                     3941.0       2551.0          0.00   
2BT1       roof                  

In [13]:
dfp.loc['3EP1'].at['backlite','Total_Good']

31943.5

In [14]:
dfp.loc['3EP1'].loc['backlite']['Total_Good']

31943.5

In [31]:
dfp

R024Glass Cons. in F.P  R124Good Prod.  \
Workcenter Window family                                                  
-139126    -163749                               3941.0          2551.0   
2BT1       -35526                                1677.0             0.0   
           Result                              -35526.0         36485.0   
           backlite                            -27081.0         28075.0   
           multi-panels 2 (back                 -8445.0          8410.0   
2BT2       -19630                                  22.0             0.0   
           Result                              -19630.0         19646.0   
           multi-panels 1 (fron                -19630.0         19646.0   
2FG8       -25851                                 850.0             0.0   
           Lamellas' sunroof 0                  -7144.0          7085.0   
           Result                              -50474.0         50648.0   
           front door                          -10710.0         11350.0   
           rear door                           -32620.0         32213.0   
3EP1       -33156                                -320.0          2252.0   
           Lamellas' sunroof 0                      0.0           -52.0   
           Result                              -33156.0         30191.0   
           backlite                            -33156.0         30243.0   
3EP3       -24963                                 -19.0          1689.0   
           Result                              -24963.0         22497.0   
           multi-panels 1 (fron                -14558.0         13191.0   
           multi-panels 2 (back                -10405.0          9306.0   
Result     -163749                             159467.0             0.0   
2BT1       roof                                 -8445.0          8410.0   
2BT2       roof                                -19630.0         19646.0   
2FG8       roof                                 -7144.0          7085.0   
3EP1       roof                                     0.0           -52.0   
3EP3       roof                                -24963.0         22497.0   

                                 In/Out Delta  Qual. Prod.  Q122Rework A  \
Workcenter Window family                                                   
-139126    -163749                        0.0        900.0        521.14   
2BT1       -35526                     36485.0          0.0        103.78   
           Result                      1677.0          0.0          0.00   
           backlite                    1677.0          0.0          0.00   
           multi-panels 2 (back           0.0          0.0          0.00   
2BT2       -19630                     19646.0          0.0        100.08   
           Result                        22.0          0.0          0.00   
           multi-panels 1 (fron          22.0          0.0          0.00   
2FG8       -25851                     50648.0          0.0        101.41   
           Lamellas' sunroof 0           18.0          0.0          0.00   
           Result                       850.0          0.0          0.00   
           front door                   821.0          0.0          0.00   
           rear door                     11.0          0.0          0.00   
3EP1       -33156                     30191.0          0.0         94.40   
           Lamellas' sunroof 0            0.0         52.0          0.00   
           Result                      -320.0       2252.0       1109.00   
           backlite                    -320.0       2200.0       1109.00   
3EP3       -24963                     22497.0          0.0         92.18   
           Result                       -19.0       1689.0        515.00   
           multi-panels 1 (fron           0.0       1107.0        371.00   
           multi-panels 2 (back         -19.0        582.0        144.00   
Result     -163749                     3941.0       2551.0          0.00   
2BT1       roof                  

In [25]:
type(dfp.loc['3EP1'].loc['backlite']['Total_Good'])

numpy.float64

In [15]:
def cycletime(a,b):
    try:
        ans = a*3600/b
        
    except:
        ans = None
        print('error occured')
    return ans
dfp['C/Time'] = dfp.apply(lambda row : cycletime(row['C005Working Hours'], row['Total_input']), axis = 1)

C:\Users\chongs\Anaconda3\envs\datacontrol\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


In [16]:
dfp

R024Glass Cons. in F.P  R124Good Prod.  \
Workcenter Window family                                                  
-139126    -163749                               3941.0          2551.0   
2BT1       -35526                                1677.0             0.0   
           Result                              -35526.0         36485.0   
           backlite                            -27081.0         28075.0   
           multi-panels 2 (back                 -8445.0          8410.0   
2BT2       -19630                                  22.0             0.0   
           Result                              -19630.0         19646.0   
           multi-panels 1 (fron                -19630.0         19646.0   
2FG8       -25851                                 850.0             0.0   
           Lamellas' sunroof 0                  -7144.0          7085.0   
           Result                              -50474.0         50648.0   
           front door                          -10710.0         11350.0   
           rear door                           -32620.0         32213.0   
3EP1       -33156                                -320.0          2252.0   
           Lamellas' sunroof 0                      0.0           -52.0   
           Result                              -33156.0         30191.0   
           backlite                            -33156.0         30243.0   
3EP3       -24963                                 -19.0          1689.0   
           Result                              -24963.0         22497.0   
           multi-panels 1 (fron                -14558.0         13191.0   
           multi-panels 2 (back                -10405.0          9306.0   
Result     -163749                             159467.0             0.0   
2BT1       roof                                 -8445.0          8410.0   
2BT2       roof                                -19630.0         19646.0   
2FG8       roof                                 -7144.0          7085.0   
3EP1       roof                                     0.0           -52.0   
3EP3       roof                                -24963.0         22497.0   

                                 In/Out Delta  Qual. Prod.  Q122Rework A  \
Workcenter Window family                                                   
-139126    -163749                        0.0        900.0        521.14   
2BT1       -35526                     36485.0          0.0        103.78   
           Result                      1677.0          0.0          0.00   
           backlite                    1677.0          0.0          0.00   
           multi-panels 2 (back           0.0          0.0          0.00   
2BT2       -19630                     19646.0          0.0        100.08   
           Result                        22.0          0.0          0.00   
           multi-panels 1 (fron          22.0          0.0          0.00   
2FG8       -25851                     50648.0          0.0        101.41   
           Lamellas' sunroof 0           18.0          0.0          0.00   
           Result                       850.0          0.0          0.00   
           front door                   821.0          0.0          0.00   
           rear door                     11.0          0.0          0.00   
3EP1       -33156                     30191.0          0.0         94.40   
           Lamellas' sunroof 0            0.0         52.0          0.00   
           Result                      -320.0       2252.0       1109.00   
           backlite                    -320.0       2200.0       1109.00   
3EP3       -24963                     22497.0          0.0         92.18   
           Result                       -19.0       1689.0        515.00   
           multi-panels 1 (fron           0.0       1107.0        371.00   
           multi-panels 2 (back         -19.0        582.0        144.00   
Result     -163749                     3941.0       2551.0          0.00   
2BT1       roof                  

In [17]:
# OEE Performance by MC

oee_dic = {'OCT':{('HNP3','BL&SR') : 10.5, ('BT2','BL&SR') : 16.7,('EPBS','BL&SR') : 10.5,('EPBS','L0') : 9,('NPBS','BL&SR') : 12.5},'Total_Good':{('HNP3','BL&SR') : dfp.loc['2BT1'].loc['Result']['Total_Good'], ('BT2','BL&SR') : dfp.loc['2BT2'].loc['Result']['Total_Good'],('EPBS','BL&SR') : dfp.loc['3EP1'].loc['Result']['Total_Good'] - dfp.loc['3EP1'].loc["Lamellas' sunroof 0"]['Total_Good'],('EPBS','L0') : dfp.loc['3EP1'].loc["Lamellas' sunroof 0"]['Total_Good'],('NPBS','BL&SR') : dfp.loc['3EP3'].loc['Result']['Total_Good']},'HU':{('HNP3','BL&SR') : dfp.loc['2BT1'].loc['Result']['C006Utilization Hours'], ('BT2','BL&SR') : dfp.loc['2BT2'].loc['Result']['C006Utilization Hours'],('EPBS','BL&SR') : dfp.loc['3EP1'].loc['Result']['C006Utilization Hours'] - dfp.loc['3EP1'].loc["Lamellas' sunroof 0"]['C006Utilization Hours'],('EPBS','L0') : dfp.loc['3EP1'].loc["Lamellas' sunroof 0"]['C006Utilization Hours'],('NPBS','BL&SR') : dfp.loc['3EP3'].loc['Result']['C006Utilization Hours']},'OAD':{('HNP3','BL&SR') : dfp.loc['2BT1'].loc['Result']['C20xOAD'], ('BT2','BL&SR') : dfp.loc['2BT1'].loc['Result']['C20xOAD'],('EPBS','BL&SR') : dfp.loc['3EP1'].loc['Result']['C20xOAD'] - dfp.loc['3EP1'].loc["Lamellas' sunroof 0"]['C20xOAD'],('EPBS','L0') : dfp.loc['3EP1'].loc["Lamellas' sunroof 0"]['C20xOAD'],('NPBS','BL&SR') : dfp.loc['3EP3'].loc['Result']['C20xOAD']},'RNDTime':{('HNP3','BL&SR') : np.nan, ('BT2','BL&SR') : np.nan,('EPBS','BL&SR') : np.nan,('EPBS','L0') : np.nan,('NPBS','BL&SR') : np.nan}}
oee = pd.DataFrame(oee_dic)

In [19]:

def oee_mc(a,b,c,d):
    try:
        ans = a*(b/3600)/(c+d)
        
    except:
        ans = None
        print('error occured')
    return ans
oee['OEE'] = oee.apply(lambda row : oee_mc(row['OCT'], row['Total_Good'], row['HU'],row['OAD']), axis = 1)

C:\Users\chongs\Anaconda3\envs\datacontrol\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in double_scalars
  after removing the cwd from sys.path.


In [20]:
oee

HU  OAD   OCT  RNDTime  Total_Good       OEE
BT2  BL&SR   90.67  0.0  16.7      NaN    19624.00  1.004010
EPBS BL&SR  124.02  0.0  10.5      NaN    31943.50  0.751238
     L0       0.00  0.0   9.0      NaN      -52.00      -inf
HNP3 BL&SR  161.08  0.0  10.5      NaN    34808.00  0.630267
NPBS BL&SR   96.00  0.0  12.5      NaN    23253.75  0.841064

In [21]:
EPBS_OEE = (10.5*oee.loc['EPBS'].loc['BL&SR']['Total_Good']*oee.loc['EPBS'].loc['BL&SR']['OEE']+9*oee.loc['EPBS'].loc['L0']['Total_Good']*oee.loc['EPBS'].loc['L0']['OEE'])/(10.5*oee.loc['EPBS'].loc['BL&SR']['Total_Good']*9*oee.loc['EPBS'].loc['L0']['OEE'])

C:\Users\chongs\Anaconda3\envs\datacontrol\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in double_scalars
  """Entry point for launching an IPython kernel.


In [22]:
EPBS_OEE

nan

In [19]:
type(oee.loc['EPBS'].loc['L0']['OEE'])

numpy.float64

### NaN은 numpy value임 type(dfp.loc['3EP1'].loc["Lamellas' sunroof 0"]['Machine_Yield'])  (np.NaN)

In [23]:
dic = {}
xs= ['2BT1','2BT2','2FG8','3EP1','3EP3']

for x in xs:
    try :
        dic[x] = {'Total_input': dfp.loc[x].loc['backlite']['Total_input'], 'Total_Good': dfp.loc[x].loc['backlite']['Total_Good'], 'Machine_Yield': dfp.loc[x].loc['backlite']['Machine_Yield']}
        print('successful')
    except:
        dic[x] = {'Total_input': np.NaN , 'Total_Good': np.NaN , 'Machine_Yield':np.NaN}
        print('error occured')
    back_p = pd.DataFrame(dic).transpose()

for x in xs:
    try :
        dic[x] = {'Total_input': dfp.loc[x].loc['roof']['Total_input'], 'Total_Good': dfp.loc[x].loc['roof']['Total_Good'], 'Machine_Yield': dfp.loc[x].loc['roof']['Machine_Yield']}
        print('successful')
    except:
        dic[x] = {'Total_input': np.NaN , 'Total_Good': np.NaN , 'Machine_Yield':np.NaN}
        print('error occured')
    roof_p = pd.DataFrame(dic).transpose()    

successful
error occured
error occured
successful
error occured
successful
successful
successful
successful
successful


In [24]:
#back lite performance by machine
back_p

,Machine_Yield,Total_Good,Total_input
2BT1,98.817100,26398.0,26714.0
2BT2,NaN,NaN,NaN
2FG8,NaN,NaN,NaN
3EP1,96.343045,31943.5,33156.0
3EP3,NaN,NaN,NaN


In [25]:
#Sunroof performance by machine
roof_p

,Machine_Yield,Total_Good,Total_input
2BT1,99.632745,8410.00,8441.0
2BT2,99.969435,19624.00,19630.0
2FG8,99.731866,7067.00,7086.0
3EP1,inf,-52.00,-0.0
3EP3,93.152866,23153.75,24963.0


In [26]:
# 전처리 후처리 별 실적

In [27]:
dfp_2 = dfp.loc['2BT1'].add(dfp.loc['2BT2'], fill_value=0)
dfp_2 = dfp_2.add(dfp.loc['2FG8'], fill_value=0).loc[["Lamellas' sunroof 0",'backlite','multi-panels 1 (fron','multi-panels 2 (back'],:]
df_2lamellas=dfp_2.loc["Lamellas' sunroof 0"]
df_2front=dfp_2.loc['multi-panels 1 (fron']
df_2rear=dfp_2.loc['multi-panels 2 (back']
new = pd.DataFrame({'Sunroof' : df_2lamellas.values+df_2front.values+df_2rear}).transpose()
df_pp = pd.concat([dfp_2,new])

df_pp['Yield'] = df_pp.apply(lambda row : machine_yield(row['Total_Good'], row['Total_input']), axis = 1)


In [28]:
dfp_3 = dfp.loc['3EP1'].add(dfp.loc['3EP3'], fill_value=0).loc[["Lamellas' sunroof 0",'backlite','multi-panels 1 (fron','multi-panels 2 (back'],:]
df_3lamellas=dfp_3.loc["Lamellas' sunroof 0"]
df_3front=dfp_3.loc['multi-panels 1 (fron']
df_3rear=dfp_3.loc['multi-panels 2 (back']
new = pd.DataFrame({'Sunroof' : df_3lamellas.values+df_3front.values+df_3rear}).transpose()
df_ff = pd.concat([dfp_3,new])
df_ff['Yield'] = df_ff.apply(lambda row : machine_yield(row['Total_Good'], row['Total_input']), axis = 1)


C:\Users\chongs\Anaconda3\envs\datacontrol\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: divide by zero encountered in double_scalars


In [29]:
df_concat = pd.concat([df_pp,df_ff])
index_out = ['PP','PP','PP','PP','PP','FF','FF','FF','FF','FF']
hier_index = list(zip(index_out,df_concat.index))
hier_index = pd.MultiIndex.from_tuples(hier_index)
df_new_work = pd.DataFrame(df_concat.values,hier_index,df_concat.columns)

In [30]:
df_new_work

R024Glass Cons. in F.P  R124Good Prod.  In/Out Delta  \
PP Lamellas' sunroof 0                  -7144.0          7085.0          18.0   
   backlite                            -27081.0         28075.0        1677.0   
   multi-panels 1 (fron                -19630.0         19646.0          22.0   
   multi-panels 2 (back                 -8445.0          8410.0           0.0   
   Sunroof                             -35219.0         35141.0          40.0   
FF Lamellas' sunroof 0                      0.0           -52.0           0.0   
   backlite                            -33156.0         30243.0        -320.0   
   multi-panels 1 (fron                -14558.0         13191.0           0.0   
   multi-panels 2 (back                -10405.0          9306.0         -19.0   
   Sunroof                             -24963.0         22445.0         -19.0   

                         Qual. Prod.  Q122Rework A  Scraps  Scrap Float  \
PP Lamellas' sunroof 0           0.0           0.0    77.0         58.0   
   backlite                      0.0           0.0   683.0        367.0   
   multi-panels 1 (fron          0.0           0.0     6.0          0.0   
   multi-panels 2 (back          0.0           0.0    35.0          4.0   
   Sunroof                       0.0           0.0   118.0         62.0   
FF Lamellas' sunroof 0          52.0           0.0     0.0          0.0   
   backlite                   2200.0        1109.0   393.0          0.0   
   multi-panels 1 (fron       1107.0         371.0   260.0          0.0   
   multi-panels 2 (back        582.0         144.0   498.0          0.0   
   Sunroof                    1741.0         515.0   758.0          0.0   

                         Scraps rework  Labour hours  C20xOAD  \
PP Lamellas' sunroof 0             0.0           0.0      0.0   
   backlite                        0.0           0.0      0.0   
   multi-panels 1 (fron            0.0           0.0      0.0   
   multi-panels 2 (back            0.0           0.0      0.0   
   Sunroof                         0.0           0.0      0.0   
FF Lamellas' sunroof 0            52.0           0.0      0.0   
   backlite                      729.0           0.0      0.0   
   multi-panels 1 (fron          736.0           0.0      0.0   
   multi-panels 2 (back          141.0           0.0      0.0   
   Sunroof                       929.0           0.0      0.0   

                         C005Working Hours  C006Utilization Hours  \
PP Lamellas' sunroof 0               23.08                  24.59   
   backlite                          82.61                 124.08   
   multi-panels 1 (fron              80.50                  90.67   
   multi-panels 2 (back              25.74                  37.00   
   Sunroof                          129.32                 152.26   
FF Lamellas' sunroof 0                0.00                   0.00   
   backlite                         104.01                 124.02   
   multi-panels 1 (fron              51.76                  57.33   
   multi-panels 2 (back              34.91                  38.67   
   Sunroof                           86.67                  96.00   

                         C10xDowntime Hours  Total_input  Current_q  \
PP Lamellas' sunroof 0                 1.51       7086.0        0.0   
   backlite                           41.47      26714.0        0.0   
   multi-panels 1 (fron               10.17      19630.0        0.0   
   multi-panels 2 (back               11.26       8441.0        0.0   
   Sunroof                            22.94      35157.0        0.0   
FF Lamellas' sunroof 0                 0.00          0.0        0.0   
   backlite                           20.01      33156.0      362.0   
   multi-panels 1 (fron                5.57      14558.0        0.0   
   multi-panels 2 (back                3.76      10405.0      297.0   
   Sunroof                             9.33      24963.0      297.0   

                         Q_to_Good  Total_Good  M

# 여기까지 함

In [25]:
# G = [['2BT1','2BT2','2FG8']]
# for X in G:
#     df1 = dfp.loc[X].xs('multi-panels 1 (fron',level='Window family').values
#     df2 = dfp.loc[X].xs('multi-panels 2 (back',level='Window family').values
#     df3 = df1 + df2
#     return df3
   
    
# new = pandas.DataFrame.from_dict({'a':{'sunroof' : df3}}

In [26]:
df_back = dfp.xs('backlite',level='Window family')
df_roof = dfp.xs('multi-panels 2 (back',level='Window family')

In [27]:
df_back

,R024Glass Cons. in F.P,R124Good Prod.,In/Out Delta,Qual. Prod.,Q122Rework A,Scraps,Scrap Float,Scraps rework,Labour hours,C005Working Hours,C10xDowntime Hours,Total_input,Current_q,Q_to_Good,Total_Good,Machine_Yield
Workcenter,,,,,,,,,,,,,,,,
2BT1,-27081,28075,1677,0,0.0,683.0,367.0,0.0,0.0,82.61,41.47,26714.0,0.0,0.0,26398.0,98.817100
3EP1,-33156,30243,-320,2200,1109.0,393.0,0.0,729.0,0.0,104.01,20.01,33156.0,362.0,271.5,31943.5,96.343045


In [28]:
dfp_2FG8 = dfp.loc['2FG8']

In [29]:
dfp_3EP1 = dfp.loc['3EP1']

In [30]:
dfp_sum = dfp_3EP1.add(dfp_2FG8, fill_value=0)

In [31]:
dfp_sum

,R024Glass Cons. in F.P,R124Good Prod.,In/Out Delta,Qual. Prod.,Q122Rework A,Scraps,Scrap Float,Scraps rework,Labour hours,C005Working Hours,C10xDowntime Hours,Total_input,Current_q,Q_to_Good,Total_Good,Machine_Yield
Window family,,,,,,,,,,,,,,,,
-25851,850.0,0.0,50648.0,0.0,101.41,529.0,0.0,529.0,141.61,18.39,0.00,-850.0,-630.41,-472.8075,-51019.3975,6002.282059
-33156,-320.0,2252.0,30191.0,0.0,94.40,0.0,781.0,0.0,104.01,20.01,0.00,-461.0,-94.40,-70.8000,-27915.4000,6055.401302
Lamellas' sunroof 0,-7144.0,7033.0,18.0,52.0,0.00,77.0,58.0,52.0,0.00,23.08,1.51,7086.0,0.00,0.0000,7015.0000,inf
Result,-83630.0,80839.0,530.0,2252.0,1109.00,1069.0,529.0,781.0,0.00,245.62,38.40,83101.0,362.00,271.5000,81689.5000,195.891887
backlite,-33156.0,30243.0,-320.0,2200.0,1109.00,393.0,0.0,729.0,0.00,104.01,20.01,33156.0,362.00,271.5000,31943.5000,96.343045
front door,-10710.0,11350.0,821.0,0.0,0.00,181.0,130.0,0.0,0.00,27.07,2.51,10580.0,0.00,0.0000,10529.0000,99.517958
rear door,-32620.0,32213.0,11.0,0.0,0.00,418.0,341.0,0.0,0.00,91.46,14.37,32279.0,0.00,0.0000,32202.0000,99.761455


In [32]:
dfp

R024Glass Cons. in F.P  R124Good Prod.  \
Workcenter Window family                                                  
-139126    -163749                                 3941            2551   
2BT1       -35526                                  1677               0   
           Result                                -35526           36485   
           backlite                              -27081           28075   
           multi-panels 2 (back                   -8445            8410   
2BT2       -19630                                    22               0   
           Result                                -19630           19646   
           multi-panels 1 (fron                  -19630           19646   
2FG8       -25851                                   850               0   
           Lamellas' sunroof 0                    -7144            7085   
           Result                                -50474           50648   
           front door                            -10710           11350   
           rear door                             -32620           32213   
3EP1       -33156                                  -320            2252   
           Lamellas' sunroof 0                        0             -52   
           Result                                -33156           30191   
           backlite                              -33156           30243   
3EP3       -24963                                   -19            1689   
           Result                                -24963           22497   
           multi-panels 1 (fron                  -14558           13191   
           multi-panels 2 (back                  -10405            9306   
Result     -163749                               159467               0   

                                 In/Out Delta  Qual. Prod.  Q122Rework A  \
Workcenter Window family                                                   
-139126    -163749                          0          900        521.14   
2BT1       -35526                       36485            0        103.78   
           Result                        1677            0          0.00   
           backlite                      1677            0          0.00   
           multi-panels 2 (back             0            0          0.00   
2BT2       -19630                       19646            0        100.08   
           Result                          22            0          0.00   
           multi-panels 1 (fron            22            0          0.00   
2FG8       -25851                       50648            0        101.41   
           Lamellas' sunroof 0             18            0          0.00   
           Result                         850            0          0.00   
           front door                     821            0          0.00   
           rear door                       11            0          0.00   
3EP1       -33156                       30191            0         94.40   
           Lamellas' sunroof 0              0           52          0.00   
           Result                        -320         2252       1109.00   
           backlite                      -320         2200       1109.00   
3EP3       -24963                       22497            0         92.18   
           Result                         -19         1689        515.00   
           multi-panels 1 (fron             0         1107        371.00   
           multi-panels 2 (back           -19          582        144.00   
Result     -163749                       3941         2551          0.00   

                                 Scraps  Scrap Float  Scraps rework  \
Workcenter Window family                                              
-139126    -163749                900.0        97.92          96.69   
2BT1       -35526                 371.0         0.00         371.00   
           Result                 718.0       371.00           0.00   
           backlite               683.0       367.00

In [33]:

df_back

,R024Glass Cons. in F.P,R124Good Prod.,In/Out Delta,Qual. Prod.,Q122Rework A,Scraps,Scrap Float,Scraps rework,Labour hours,C005Working Hours,C10xDowntime Hours,Total_input,Current_q,Q_to_Good,Total_Good,Machine_Yield
Workcenter,,,,,,,,,,,,,,,,
2BT1,-27081,28075,1677,0,0.0,683.0,367.0,0.0,0.0,82.61,41.47,26714.0,0.0,0.0,26398.0,98.817100
3EP1,-33156,30243,-320,2200,1109.0,393.0,0.0,729.0,0.0,104.01,20.01,33156.0,362.0,271.5,31943.5,96.343045
